# Tópicos

1. Build the Neural Network

2. Get Device for Training

3. Definição da Classe ```nn.Module```

4. Camadas do Modelo
    * nn.Flatten
    * nn.Linear5
    * nn.ReLU
    * nn.Sequantial
    * nn.Softmax

5. Parâmetros do Modelo

# Build the Neural Network

Redes Neurais incluem camadas e módulos que realizam operações em dados.

O módulo [torch.nn](https://pytorch.org/docs/stable/nn.html) fornece as funções necessárias para contribuir uma rede neural. Todo módulo do PyTorch é uma subclasse do [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). Uma rede neural é um módulo que consiste de outros módulos (camadas).

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Get Device for Training

O treinamento deve ser realizado preferencialmente de forma acelerada (CUDA, MPS, MTIA ou XPS).

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


# Definição da classe nn.Module

A definição da rede neural através da subclasse ```nn.Module``` inicializa as camadas da rede neural no ```__init__```. Toda subclasse de ```nn.Module``` implementa operações nos dados de entrada com o método ```forward```.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Para problemas complexos, os quais não podem ser resolvidos por regressão linear, há necessidade de introdução de não linearidades. Por este motivo, deve-se utilizar uma função de ativação após cada transformação linear.

Se a função de ativação for aplicada somente ao final, as camdas ocultas serão desnecessariamente utilizadas, devido a possibilidade de reduzir várias aplicações de funções lineares em um única.

Uma instância de ```NeuralNetwork``` pode ser criada e movida para o ```device```. A estrutura pode ser exibida.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Para usar o modelo, os dados de entradas serõa inseridos. Isto executará o ```forward```, junto de operações auxiliares. [Background Operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).

Chamar o modelo na entrada retornará um tensor de 2 dimensões, correspondentes a cada um dos 10 tipos de saída com os valroes de predição de cada classe.

O valor de predição será obtido por ```nn.Softmax```.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8], device='cuda:0')


# Camadas do Modelo

Exemplo FashionMNIST

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


#### nn.Flatten

Inicializar o [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) converte cada imagem 2D (28x28) em um vetor de pixels de tamanho 784.

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear

A camada [nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) aplica a transformação linear de pesos e limiares nos dados de entrada

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU

Ativações não lineares são mapas complexos entre as entradas e saídas. Elas são aplicadas após uma transformação linear para introduzir não linearidades.

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.4095,  0.0224, -0.4912, -0.1153, -0.2502, -0.3839, -0.2982,  0.0884,
         -0.3399,  0.3616,  0.4145, -0.0483,  0.0333, -0.0562, -0.3786, -0.0926,
         -0.1248, -0.6659, -0.0558,  0.4460],
        [-0.2827, -0.5175, -0.3282, -0.2706, -0.1217, -0.2572, -0.4950, -0.1868,
         -0.3092,  0.3189,  0.2988,  0.0385, -0.2925,  0.1394, -0.1160, -0.2565,
          0.4079, -0.4372,  0.3396,  0.2350],
        [-0.3873, -0.3934, -0.1815, -0.3436, -0.2814, -0.3822, -0.4340,  0.0370,
         -0.0795,  0.3878,  0.4614,  0.1842,  0.0146,  0.4319, -0.2141, -0.1272,
         -0.0272, -0.7625,  0.5143,  0.3402]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0224, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0884, 0.0000,
         0.3616, 0.4145, 0.0000, 0.0333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.4460],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.3189, 0.2988, 0.0385, 0.0000, 0.1394, 0.00

#### nn.Sequential

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) contém módulos de ordenação. Os dados são passados por todos módulos na mesma ordem como definido.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

#### nn.Softmax

A última camada linear de uma rede neural retorna *logits* entre [-infty, infty] que são passadas pela [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html). Os *logits* são escalados entre [0, 1] representando o modelo de predição probabilística do modelo para cada classe.

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

# Parâmetros do Modelo

Muitas camadas dentro de uma rede neural são parametrizadas (ajustados durante treinamento). A criação de uma subclasse de ```nn.Module``` e mapeam todos campos dentro de um modelo e tornam esse parâmetros acessíveis com ```parameters()``` ou ```named_parameters()```.

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0161,  0.0299,  0.0294,  ..., -0.0190,  0.0181, -0.0166],
        [ 0.0012,  0.0192,  0.0062,  ..., -0.0038,  0.0150,  0.0342]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0138, -0.0087], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0318, -0.0034,  0.0185,  ..., -0.0384, -0.0209, -0.0331],
        [-0.0434, -0.0026,  0.0072,  ..., -0.0041,  0.0341,  0.0007]],
       device='cuda:0', grad_fn=<Sl